<a href="https://colab.research.google.com/github/kk78-kx/TestAgent/blob/main/%E0%B9%81%E0%B8%9A%E0%B8%9A%E0%B8%9D%E0%B8%B6%E0%B8%81%E0%B8%AB%E0%B8%B1%E0%B8%94%E0%B9%80%E0%B8%AA%E0%B8%A3%E0%B8%B4%E0%B8%A1%E0%B8%AA%E0%B8%B3%E0%B8%AB%E0%B8%A3%E0%B8%B1%E0%B8%9A%E0%B8%9D%E0%B8%B6%E0%B8%81%E0%B8%84%E0%B8%B4%E0%B8%94%E0%B9%81%E0%B8%A5%E0%B8%B0%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B9%80%E0%B8%A1%E0%B8%B4%E0%B8%99%E0%B8%9C%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. เขียนโค้ดให้ Agent ดูดฝุ่นเมื่อห้องสกปรก และย้ายถ้าห้องสะอาด**

In [7]:
class World:
    def __init__(self):
        # สถานะเริ่มต้นของแต่ละห้อง
        self.room_state = {"A": "Dirty", "B": "Clean"}

    def dirty(self, room):
        return self.room_state[room] == "Dirty"

    def wash(self, room):
        self.room_state[room] = "Clean"


class AutoCleaner:
    def __init__(self, world):
        self.world = world
        self.current_room = "A"  # จุดเริ่มต้น

    def sense(self):
        return {"room": self.current_room, "dirty": self.world.dirty(self.current_room)}

    def perform(self):
        info = self.sense()

        if info["dirty"]:
            print(f"👉 กำลังทำความสะอาดห้อง {info['room']} ...")
            self.world.wash(info["room"])
            action = "Clean"
        else:
            if self.current_room == "A":
                self.current_room = "B"
                action = "Move to B"
            else:
                self.current_room = "A"
                action = "Move to A"

            print(f"➡️ ห้องสะอาดแล้ว → {action}")

        return action


# --- ทดสอบการทำงาน ---
env = World()
robot = AutoCleaner(env)

for i in range(4):
    print(f"\n▶️ รอบที่ {i+1} : สถานะห้อง A={env.room_state['A']}, B={env.room_state['B']}")
    robot.perform()

print("\n📌 สถานะสุดท้าย:", env.room_state)



▶️ รอบที่ 1 : สถานะห้อง A=Dirty, B=Clean
👉 กำลังทำความสะอาดห้อง A ...

▶️ รอบที่ 2 : สถานะห้อง A=Clean, B=Clean
➡️ ห้องสะอาดแล้ว → Move to B

▶️ รอบที่ 3 : สถานะห้อง A=Clean, B=Clean
➡️ ห้องสะอาดแล้ว → Move to A

▶️ รอบที่ 4 : สถานะห้อง A=Clean, B=Clean
➡️ ห้องสะอาดแล้ว → Move to B

📌 สถานะสุดท้าย: {'A': 'Clean', 'B': 'Clean'}


## **2. เปลี่ยนสถานะเริ่มต้นให้ห้องทั้งสอง “สกปรก” แล้วทดสอบ Agent**

In [8]:
class HomeArea:
    def __init__(self):
        # ตั้งค่าสถานะเริ่มต้นเป็นสกปรกทั้งสองห้อง
        self.room = {"A": "Dirty", "B": "Dirty"}

    def check_dirty(self, pos):
        return self.room[pos] == "Dirty"

    def tidy(self, pos):
        self.room[pos] = "Clean"


class ReflexCleanerAgent:
    def __init__(self, environment):
        self.env = environment
        self.position = "A"   # จุดเริ่มต้นของหุ่นยนต์

    def sense_environment(self):
        return self.position, self.env.check_dirty(self.position)

    def do_action(self):
        location, dirty = self.sense_environment()

        if dirty:
            act = "Clean (Suck)"
            self.env.tidy(location)
            print(f"🧹 {act} → ห้อง {location} สะอาดแล้ว")
        else:
            # ถ้าห้องสะอาดแล้วให้ย้ายตำแหน่ง
            if location == "A":
                self.position = "B"
                act = "Move to B"
            else:
                self.position = "A"
                act = "Move to A"

            print(f"➡️ ห้องสะอาด → ดำเนินการ: {act}")

        return act


In [10]:
# --- ทดสอบการทำงาน ---
env = HomeArea()
agent = ReflexCleanerAgent(env)

for step in range(4):
    print(f"\nStep {step + 1}: Room A={env.room['A']}, Room B={env.room['B']}")
    agent.do_action()

print("\nFinal status:", env.room)



Step 1: Room A=Dirty, Room B=Dirty
🧹 Clean (Suck) → ห้อง A สะอาดแล้ว

Step 2: Room A=Clean, Room B=Dirty
➡️ ห้องสะอาด → ดำเนินการ: Move to B

Step 3: Room A=Clean, Room B=Dirty
🧹 Clean (Suck) → ห้อง B สะอาดแล้ว

Step 4: Room A=Clean, Room B=Clean
➡️ ห้องสะอาด → ดำเนินการ: Move to A

Final status: {'A': 'Clean', 'B': 'Clean'}


## **3. อธิบายข้อดีข้อเสียของ Simple Reflex Agent**

### **ข้อดีของ Simple Reflex Agent**


* โครงสร้างง่ายและออกแบบไม่ซับซ้อน
*    ตอบสนองเร็วเพราะทำงานตามเงื่อนไขทันที
* ใช้ทรัพยากรในการประมวลผลต่ำ
*  เหมาะกับงานหรือสภาพแวดล้อมที่กฎชัดเจนและไม่ซับซ้อนหมาะกับงานหรือสภาพแวดล้อมที่กฎชัดเจนและไม่ซับซ้อน


### **ข้อเสียของ Simple Reflex Agent**

*  ไม่มีหน่วยความจำทำให้ไม่สามารถเรียนรู้หรือจดจำสถานการณ์ที่ผ่านมาได้
*   ไม่เหมาะกับสภาพแวดล้อมที่ซับซ้อนหรือเปลี่ยนแปลงได้
*   ไม่เหมาะกับสภาพแวดล้อมที่ซับซ้อนหรือเปลี่ยนแปลงได้
*   ความยืดหยุ่นต่ำ ต้องเพิ่มหรือปรับกฎเองเมื่อเงื่อนไขเปลี่ยน
*  พึ่งพาการรับรู้ปัจจุบันเท่านั้น หากข้อมูลไม่ครบอาจทำงานผิดพลาด


## **4. เขียนคำสั่ง Python ตรวจสอบเวอร์ชันไลบรารีที่ติดตั้งใน Colab**

In [11]:
import numpy as np
import sklearn

print("NumPy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)

NumPy version: 2.0.2
Scikit-learn version: 1.6.1
